In [1]:
import os
import csv
import pandas as pd
import torch
import re
import nltk
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, matthews_corrcoef
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback, XLNetTokenizer, XLNetForSequenceClassification
import torch.nn as nn
import wandb

from google.colab import auth
from google.colab import drive
from google.colab import userdata

In [2]:
# =========================
# STEP 0: mount to drive
# =========================
#auth.authenticate_user()
#drive.mount('/content/drive')
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [3]:
# =========================
# STEP 1: Initialize WandB
# =========================
# Retrieve API Key from Colab Secrets
wandb_api_key = userdata.get('WANDB_API_KEY')

if wandb_api_key:
    os.environ["WANDB_API_KEY"] = wandb_api_key
    wandb.login(key=wandb_api_key)
    print("✅ WandB Logged in Securely")
else:
    print("❌ Error: WANDB_API_KEY not found. Set it in Colab Secrets.")


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: aalhaizaey (abdulrahim-alhaizaey). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


✅ WandB Logged in Securely


In [4]:
# =========================
# STEP 2: LOAD DATA & CLEAN TEXT
# =========================
nltk.download('stopwords')
nltk.download('wordnet')

multi_data = pd.read_csv("/content/10006_dataset_Multi.csv", encoding='ISO-8859-1')

def clean_text(text):
    if pd.isnull(text):
        return ""
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text).lower()
    return text

multi_data['cleaned_text'] = multi_data['text'].apply(clean_text)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [5]:
# =========================
# STEP 3: PREPARE DATASETS & LABEL ENCODING (Train Only)
# =========================
multi_train, multi_val = train_test_split(multi_data, test_size=0.2, stratify=multi_data['label'], random_state=42)

# Fit LabelEncoder only on training data to prevent leakage
multi_le = LabelEncoder()
multi_le.fit(multi_train['label'])
multi_train['encoded_label'] = multi_le.transform(multi_train['label'])
multi_val['encoded_label'] = multi_le.transform(multi_val['label'])


In [6]:
# =========================
# STEP 4: TOKENIZATION (Train Data Only for max_length)
# =========================
tokenizer = XLNetTokenizer.from_pretrained("xlnet-base-cased")

# Calculate max_length using training data
max_length = min(tokenizer.model_max_length, max(multi_train['cleaned_text'].apply(lambda x: len(tokenizer.tokenize(x)))))
print(f"Using max_length from training set only: {max_length}")

def tokenize_data(df, label_col):
    encodings = tokenizer(df['cleaned_text'].tolist(), truncation=True, padding=True, max_length=max_length, return_tensors="pt")
    labels = torch.tensor(df[label_col].tolist())
    return encodings, labels

train_multi_enc, train_multi_labels = tokenize_data(multi_train, 'encoded_label')
val_multi_enc, val_multi_labels = tokenize_data(multi_val, 'encoded_label')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/798k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

Using max_length from training set only: 125


In [7]:
# =========================
# STEP 5: DATASET CLASS & METRICS
# =========================
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted', zero_division=0)
    mcc = matthews_corrcoef(labels, predictions)
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "mcc": mcc
    }


In [8]:
# =========================
# STEP 6: FOCAL LOSS & CUSTOM TRAINER
# =========================
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0, reduction='mean'):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        ce_loss = nn.CrossEntropyLoss(reduction='none')(inputs, targets)
        pt = torch.exp(-ce_loss)
        focal_loss = self.alpha * (1 - pt) ** self.gamma * ce_loss
        return focal_loss.mean() if self.reduction == 'mean' else focal_loss.sum()

class CustomTrainer(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.training_loss = []
        self.validation_loss = []
        self.results = []

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fn = FocalLoss(alpha=0.25, gamma=2.0)
        loss = loss_fn(logits, labels)
        return (loss, outputs) if return_outputs else loss

    def evaluate(self, *args, **kwargs):
        output = super().evaluate(*args, **kwargs)
        self.validation_loss.append(output['eval_loss'])
        self.results.append(output.copy())
        return output

    def log(self, logs, *args, **kwargs):
        super().log(logs, *args, **kwargs)
        if 'loss' in logs:
            self.training_loss.append(logs['loss'])


In [9]:
# =========================
# STEP 7: ADD PLOTTING (Confusion Matrices & Loss Curves)
# =========================
def plot_confusion_matrix(labels, predictions, label_encoder, output_dir, run_name):
    class_names = label_encoder.classes_
    conf_matrix = confusion_matrix(labels, predictions)
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.savefig(f"{output_dir}/{run_name}_confusion_numbers.png")
    plt.close()

def plot_confusion_matrix_percent(labels, predictions, label_encoder, output_dir, run_name):
    class_names = label_encoder.classes_
    conf_matrix = confusion_matrix(labels, predictions, normalize='true')
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt='.2f', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.savefig(f"{output_dir}/{run_name}_confusion_percent.png")
    plt.close()

def plot_confusion_matrix_class_weighted(labels, predictions, label_encoder, output_dir, run_name):
    conf_matrix = confusion_matrix(labels, predictions, normalize='true')
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt='.2f', cmap='coolwarm', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.savefig(f"{output_dir}/{run_name}_confusion_weighted.png")
    plt.close()

def plot_loss(training_loss, validation_loss, output_dir, run_name):
    min_length = min(len(training_loss), len(validation_loss))
    training_loss = training_loss[:min_length]
    validation_loss = validation_loss[:min_length]
    epochs = range(1, min_length + 1)
    plt.figure(figsize=(8, 6))
    plt.plot(epochs, training_loss, label='Training Loss', marker='o')
    plt.plot(epochs, validation_loss, label='Validation Loss', marker='s', linestyle='--')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    plt.savefig(f"{output_dir}/{run_name}_loss_curves.png")
    plt.close()


In [10]:
# =========================
# STEP 8: TRAIN FUNCTION
# =========================
def train_xlnet(train_dataset, eval_dataset, num_labels, output_dir, label_encoder, config=None):
    with wandb.init(config=config):
        config = wandb.config
        run_name = wandb.run.name

        base_dir = "/content/drive/MyDrive/XLNet_8_Results"
        run_dir = os.path.join(base_dir, run_name)
        os.makedirs(run_dir, exist_ok=True)

        model = XLNetForSequenceClassification.from_pretrained(
          "xlnet-base-cased",
           num_labels=num_labels,
           dropout=config.dropout
        )


        #num_devices = torch.cuda.device_count()  # Will be 1 on Colab
        #examples_per_step = config.batch_size * num_devices * config.gradient_accumulation_steps
        #total_steps = int(np.ceil(len(train_dataset) / examples_per_step) * config.num_train_epochs)

        num_devices = torch.cuda.device_count()  # Will be 1 on Colab
        gradient_accumulation_steps = config.get('gradient_accumulation_steps', 1)  # Provide a default value
        examples_per_step = config.batch_size * num_devices * gradient_accumulation_steps
        total_steps = int(np.ceil(len(train_dataset) / examples_per_step) * config.num_train_epochs)


        warmup_steps = int(0.1 * total_steps)
        print(f"Total Steps: {total_steps}, Warmup Steps (10%): {warmup_steps}")

        training_args = TrainingArguments(
            output_dir=output_dir,
            run_name=run_name,
            num_train_epochs=config.num_train_epochs,
            per_device_train_batch_size=config.batch_size,
            per_device_eval_batch_size=config.batch_size,
            learning_rate=config.learning_rate,
            warmup_steps=warmup_steps,
            weight_decay=config.weight_decay,
            evaluation_strategy="epoch",
            logging_strategy="epoch",
            save_strategy="epoch",
            fp16=True,
            #gradient_checkpointing=True,
            save_total_limit=2,
            gradient_accumulation_steps = config.get('gradient_accumulation_steps', 1),  # Provide a default value
            #gradient_accumulation_steps=config.gradient_accumulation_steps,
            max_grad_norm=config.max_grad_norm,
            lr_scheduler_type="cosine_with_restarts",
            load_best_model_at_end=True,
            metric_for_best_model="eval_loss",
            report_to="wandb"
        )

        trainer = CustomTrainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=eval_dataset,
            compute_metrics=compute_metrics,
            callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
        )

        trainer.train()

        # Save Best Model
        best_model_path = os.path.join(run_dir, "best_model")
        trainer.save_model(best_model_path)
        print(f"✅ Best model saved to {best_model_path}")

        # Evaluate Best Model
        results = trainer.evaluate(eval_dataset=eval_dataset)

        predictions_obj = trainer.predict(eval_dataset)
        predictions = np.argmax(predictions_obj.predictions, axis=1)
        labels = predictions_obj.label_ids

        wandb.log({
            "eval_loss": results.get("eval_loss"),
            "eval_accuracy": results.get("eval_accuracy"),
            "eval_precision": results.get("eval_precision"),
            "eval_recall": results.get("eval_recall"),
            "eval_f1": results.get("eval_f1"),
            "eval_mcc": results.get("eval_mcc")
        })

        plot_confusion_matrix(labels, predictions, label_encoder, run_dir, run_name)
        plot_confusion_matrix_percent(labels, predictions, label_encoder, run_dir, run_name)
        plot_confusion_matrix_class_weighted(labels, predictions, label_encoder, run_dir, run_name)
        plot_loss(trainer.training_loss, trainer.validation_loss, run_dir, run_name)

        # Save run parameters and results to a CSV file
        results_csv = os.path.join(run_dir, "results.csv")
        with open(results_csv, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["Parameter", "Value"])
            for key, value in config.items():
                writer.writerow([key, value])
            writer.writerow(["Epoch", "Training Loss", "Validation Loss", "Accuracy", "Precision", "Recall", "F1", "Mcc"])
            for epoch, result in enumerate(trainer.results, start=1):
                writer.writerow([
                    epoch,
                    trainer.training_loss[epoch - 1] if epoch - 1 < len(trainer.training_loss) else None,
                    trainer.validation_loss[epoch - 1] if epoch - 1 < len(trainer.validation_loss) else None,
                    result.get("eval_accuracy"),
                    result.get("eval_precision"),
                    result.get("eval_recall"),
                    result.get("eval_f1"),
                    result.get("eval_mcc")
                ])

        return results


In [11]:
# =========================
# STEP 9: RUN SWEEP CONFIGURATION
# =========================
train_dataset = TextDataset(train_multi_enc, train_multi_labels)
val_dataset = TextDataset(val_multi_enc, val_multi_labels)

sweep_config = {
    'method': 'bayes',
    'metric': {'name': 'eval_loss', 'goal': 'minimize'},
    'parameters': {
        'num_train_epochs': {'values': [12]},
        'learning_rate': {'min': 1e-5, 'max': 5e-5},
        'batch_size': {'values': [8, 16, 32]},
        'weight_decay': {'min': 0.01, 'max': 0.05},
        'dropout': {'min': 0.05, 'max': 0.5},
        #'gradient_accumulation_steps': {'values': [2, 4]},
        'gradient_accumulation_steps': {'values': [2, 4]},  # Ensure this line is present
        'max_grad_norm': {'values': [1.0, 2.0]}
    },
}

In [12]:


sweep_id = wandb.sweep(sweep=sweep_config, project='XLNet_8')

wandb.agent(sweep_id, function=lambda: train_xlnet(
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    num_labels=len(multi_le.classes_),
    output_dir="multi_output",
    label_encoder=multi_le
))


Create sweep with ID: 91ggabyj
Sweep URL: https://wandb.ai/abdulrahim-alhaizaey/XLNet_8/sweeps/91ggabyj


wandb: Agent Starting Run: 2mlv5ueb with config:
wandb: 	batch_size: 32
wandb: 	dropout: 0.39393678929202386
wandb: 	gradient_accumulation_steps: 4
wandb: 	learning_rate: 2.2571946025271295e-05
wandb: 	max_grad_norm: 2
wandb: 	num_train_epochs: 12
wandb: 	weight_decay: 0.02214346734807081


pytorch_model.bin:   0%|          | 0.00/467M [00:00<?, ?B/s]

Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Total Steps: 756, Warmup Steps (10%): 75


...

wandb: WARNING Config item 'dropout' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'gradient_accumulation_steps' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'weight_decay' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'max_grad_norm' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'num_train_epochs' was locked by 'sweep' (ignored update).


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,Mcc
1,0.555000,0.049509,0.859141,0.878115,0.859141,0.862771,0.755823
2,0.184500,0.051220,0.871628,0.888721,0.871628,0.873229,0.781133
3,0.089800,0.042263,0.901099,0.905101,0.901099,0.900138,0.826570
4,0.046900,0.047064,0.903596,0.905577,0.903596,0.902081,0.826479
5,0.029900,0.045765,0.898102,0.912210,0.898102,0.902091,0.828431
6,0.014300,0.036862,0.920579,0.923622,0.920579,0.921129,0.860386
7,0.006900,0.041853,0.927073,0.927057,0.927073,0.926648,0.869788
8,0.001800,0.041570,0.927572,0.928420,0.927572,0.927677,0.871379
9,0.000900,0.041938,0.928571,0.929977,0.928571,0.929054,0.874051


✅ Best model saved to /content/drive/MyDrive/XLNet_8_Results/desert-sweep-20/best_model


eval/accuracy,▁▂▅▅▅▇███▇
eval/f1,▁▂▅▅▅▇███▇
eval/loss,▇█▄▆▅▁▃▃▃▁
eval/mcc,▁▂▅▅▅▇███▇
eval/precision,▁▂▅▅▆▇███▇
eval/recall,▁▂▅▅▅▇███▇
eval/runtime,▃▇▁▆▂▅▃▃▅█
eval/samples_per_second,▆▂█▃▇▄▆▆▄▁
eval/steps_per_second,▆▂█▃▇▄▆▆▄▁
eval_accuracy,▁
eval_f1,▁


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
